In [1]:
import tensorflow as tf
import numpy as np

### 定义TextRNN结构，使用双向的LSTM

In [2]:
class TextRNN:
    def __init__(self, batch_size, num_classes, vocab_size, sentence_len, embed_size, 
                 learning_rate, decay_steps, decay_rate, is_training):
        #1.定义超参数
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.vocab_size = vocab_size
        self.sentence_len = sentence_len
        self.embed_size = embed_size
        self.hidden_size = embed_size #lstm层的维度
        self.learning_rate = learning_rate
        self.is_training = is_training
        self.initializer = tf.random_normal_initializer(stddev=0.1)
        
        #epoch信息
        self.global_step = tf.Variable(0, trainable=False, name='global_step')
        self.epoch_step = tf.Variable(0, trainable=False, name='epoch_step')
        self.epoch_increment = tf.assign(self.epoch_step, tf.add(self.epoch_step, tf.constant(1)))
        self.decay_steps, self.decay_rate = decay_steps, decay_rate
        
        #2.输入
        self.input_x = tf.placeholder(tf.int32, [None, sentence_len], 'input_x')
#         self.input_y = tf.placeholder(tf.int32, [None], 'input_y') #单个标签
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], 'input_y') #多个标签
        self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')
        
        #3.初始化全连接层参数
        self.init_weight()
        
        #4.网络结构
        self.logits = self.inference() #[batch_size, num_classes]
        
        #5.损失函数
        self.loss_val = self.loss()
        
        #6.优化器
        self.train_op = self.train()
        
    def init_weight(self):
        self.Embedding = tf.get_variable('Embedding', [self.vocab_size, self.embed_size], dtype=tf.float32)
        self.W = tf.get_variable('W', [self.hidden_size * 2, self.num_classes], dtype=tf.float32) #双向LSTM，输出concat，所以此处为2倍
        self.b = tf.get_variable('b', [self.num_classes], dtype=tf.float32)
        
    def inference(self):
        # a.embedding
        self.sentence_embed = tf.nn.embedding_lookup(self.Embedding, self.input_x) #[batch_size, sentence_len, embed_size]
        
        # b.bidiretional lstm
        self.fw_cell = tf.contrib.rnn.BasicLSTMCell(self.hidden_size) #前向单元
        self.bw_cell = tf.contrib.rnn.BasicLSTMCell(self.hidden_size) #后向单元
#         if self.dropout_keep_prob is not None:
#             self.fw_cell = tf.contrib.rnn.DropoutWrapper(self.fw_cell, output_keep_prob=) 
                #input_keep_prob是对输入而言，output_keep_prb是对lstm各层而言
        outputs, _ = tf.nn.bidirectional_dynamic_rnn(self.fw_cell, self.bw_cell, self.sentence_embed, dtype=tf.float32)
        #输入为 [batch_size, sentence_len, embed_size]，输出为大小为2的元组，每个元素为[batch_size, sentence_len, hidden_size]
        
        # c.concat
        fw_output = outputs[0][:,-1,:]
        bw_output = outputs[1][:,-1,:] #[batch_size, 1, hidden_size]
        final_output = tf.concat([fw_output, bw_output], axis=1) #[batch_size, 1, hidden_size*2]
        final_output = tf.reshape(final_output, [-1, self.hidden_size*2]) #[batch_size, hidden_size * 2]
        
        # d.full_connection
        logits = tf.matmul(final_output, self.W) + self.b #[batch_size, num_classes]
        return logits
    
    def loss(self, l2_lambda=0.0001):
#         loss1 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.input_y, logits=self.logits)
        #先将label转化为one-hot形式，再对logits计算softmax，最后计算交叉熵
        loss1 = tf.nn.sigmoid_cross_entropy_with_logits(labels=self.input_y, logits=self.logits)
        loss1 = tf.reduce_mean(tf.reduce_sum(loss1, axis=1))
        loss2 = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()]) * l2_lambda
        return loss1 + loss2
    
    def train(self):
        learning_rate = tf.train.exponential_decay(self.learning_rate, self.global_step, self.decay_steps, self.decay_rate, staircase=True)
        train_op = tf.contrib.layers.optimize_loss(self.loss_val, self.global_step, learning_rate, optimizer='Adam')
        return train_op
    

### 测试

In [3]:
def test():
    num_classes=5
    learning_rate=0.01
    batch_size=5
    decay_step=1000
    decay_rate=0.9
    sequence_length=5
    vocab_size=10000
    embed_size=100
    is_training=True
    dropout_keep_prob=0.5
    
    model = TextRNN(batch_size, num_classes, vocab_size, sequence_length, embed_size, 
                     learning_rate, decay_step, decay_rate, True)
    print(tf.trainable_variables())
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        input_x = np.random.randint(0,100,size=(batch_size, sequence_length),dtype=np.int32)
        input_y = np.random.randint(0, 2,size=(batch_size, num_classes), dtype=np.int32)
        for i in range(20):
            #input_x = np.zeros((batch_size, sequence_length), dtype=np.int32)
            #input_y = np.array([1,0,1,1,1,2,1,1], dtype=np.int32)
            loss, logits, _ = sess.run([model.loss_val, model.logits, model.train_op],
                                            feed_dict={model.input_x: input_x, model.input_y: input_y,
                                                       model.dropout_keep_prob: dropout_keep_prob})
            logits = np.argsort(logits)
            print('****label****\n', input_y)
            print('****pre_y****\n', logits, '\n')

In [4]:
tf.reset_default_graph()
test()

[<tf.Variable 'Embedding:0' shape=(10000, 100) dtype=float32_ref>, <tf.Variable 'W:0' shape=(200, 5) dtype=float32_ref>, <tf.Variable 'b:0' shape=(5,) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/fw/basic_lstm_cell/kernel:0' shape=(200, 400) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/fw/basic_lstm_cell/bias:0' shape=(400,) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/bw/basic_lstm_cell/kernel:0' shape=(200, 400) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/bw/basic_lstm_cell/bias:0' shape=(400,) dtype=float32_ref>]
****label****
 [[1 0 0 1 1]
 [0 0 1 0 0]
 [0 1 1 0 0]
 [1 1 1 1 1]
 [1 0 1 1 0]]
****pre_y****
 [[4 0 1 2 3]
 [4 0 1 2 3]
 [4 0 1 2 3]
 [4 0 1 2 3]
 [4 0 1 2 3]] 

****label****
 [[1 0 0 1 1]
 [0 0 1 0 0]
 [0 1 1 0 0]
 [1 1 1 1 1]
 [1 0 1 1 0]]
****pre_y****
 [[4 1 0 2 3]
 [4 0 1 3 2]
 [4 0 1 3 2]
 [4 0 1 3 2]
 [4 1 0 3 2]] 

****label****
 [[1 0 0 1 1]
 [0 0 1 0 0]
 [0 1 1 0 0]
 [1 1 1 1 1]
 [1 0 1 1 0]]
****pre_y****
 [[4 1 0 2 3]
 [4 1 0 3 2]

### 训练

In [5]:
import sys
import tensorflow as tf
import numpy as np
# from tflearn.data_utils import to_categorical, pad_sequences
import os
import pickle
import h5py

In [6]:
#定义超参数
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('batch_size', 256, 'batch_size')
tf.app.flags.DEFINE_integer('num_classes', 1999, 'num_classes')
tf.app.flags.DEFINE_integer('sentence_len', 100, 'length of each sentence')
tf.app.flags.DEFINE_integer('embed_size', 100, 'embedding size')
tf.app.flags.DEFINE_float('learning_rate', 0.01, '')
tf.app.flags.DEFINE_float('decay_rate', 0.8, '')
tf.app.flags.DEFINE_integer('decay_steps', 1000, 'number of steps before decay learning rate')
tf.app.flags.DEFINE_bool('is_training', True, '')

tf.app.flags.DEFINE_integer('num_epoch', 10, 'number of epoch')

tf.app.flags.DEFINE_string("ckpt_dir","testrnn_multilabel_checkpoint/","checkpoint location for the model")
tf.app.flags.DEFINE_string("cache_path","textrnn_multilabel_checkpoint/data_cache.pik","data chche for the model")

In [7]:
import time
def log(str):
    t = time.localtime()
    print("[%4d/%02d/%02d %02d:%02d:%02d]"%(t.tm_year, t.tm_mon, t.tm_mday, t.tm_hour, t.tm_min, t.tm_sec), end=' ')
    print(str)

In [8]:
def main(_):
    #1.加载数据
    base_path = '/data/chenhy/data/ieee_zhihu_cup/'
    cache_file_h5py = base_path + 'data.h5'
    cache_file_pickle = base_path + 'vocab_label.pik'
    word2index,label2index,train_X,train_y,vaild_X,valid_y,test_X,test_y = load_data(cache_file_h5py, cache_file_pickle)
    vocab_size = len(word2index)
    index2word = {index: word for word, index in word2index.items()}
    index2label = {index: label for label, index in label2index.items()}

    print("train_X[0:5]:", train_X[0:5])
    print("train_Y[0:5]:", train_y[0:5])
    train_y_short = get_target_label_short(train_y[0])
    print("train_y_short:", train_y_short)
    
    #2.创建session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        model = TextRNN(FLAGS.batch_size, FLAGS.num_classes, vocab_size, FLAGS.sentence_len,FLAGS.embed_size, 
                        FLAGS.learning_rate, FLAGS.decay_steps, FLAGS.decay_rate, FLAGS.is_training)
        saver = tf.train.Saver()
        batch_size = FLAGS.batch_size
        CONTINUE_TRAIN = True
        if os.path.exists(FLAGS.ckpt_dir + 'checkpoint'):
            log("restore from checkpoint")
            saver.restore(sess, tf.train.latest_checkpoint(FLAGS.ckpt_dir))
            if CONTINUE_TRAIN: log("continue training...")
        else:
            log('init variables')
            sess.run(tf.global_variables_initializer())
#             #是否使用embedding
#             print('assign pre-trained embedding')
#             embedding_assign = tf.assign(model.Embedding, tf.constant(np.array(embedding_final))) #为model.Embedding赋值
#             sess.run(embedding_assign)
        if not os.path.exists(FLAGS.ckpt_dir + 'checkpoint') or CONTINUE_TRAIN:
            num_of_data = len(train_y)
            for _ in range(FLAGS.num_epoch):
                epoch = sess.run(model.epoch_step)
                loss, counter = 0., 0.
                for start, end in zip(range(0, num_of_data, batch_size), range(batch_size, num_of_data, batch_size)):
                    loss_tmp,  _ = sess.run([model.loss_val, model.train_op], 
                                                    feed_dict={model.input_x: train_X[start:end, :FLAGS.sentence_len], model.input_y: train_y[start:end],
                                                               model.dropout_keep_prob: 1})
                    loss, counter = loss + loss_tmp, counter + 1
                    if counter % 200 == 0:
                        log("Epoch %d\Batch %d\ Train Loss:%.3f"%(epoch, counter, loss/float(counter)))
                    if counter % 1400 == 0:
                        print('run model on validation data...')
                        loss_valid, f1_score, precision, recall = do_eval(sess, model, vaild_X, valid_y)
                        log("Epoch %d/ Validation Loss:%.3f/ F1_score:%.3f/ Precision:%.3f/ Recall:%.3f"%(epoch, loss_valid, f1_score, precision, recall))
                        #save the checkpoint
                        save_path = FLAGS.ckpt_dir + 'model.ckpt'
                        saver.save(sess, save_path, global_step=model.epoch_step)
                sess.run(model.epoch_increment)
            
    

In [9]:
#一些辅助函数
def get_target_label_short(eval_y):
    res = [idx for idx in range(len(eval_y)) if eval_y[idx] > 0] #结果如：[45,100,1555]
    return res

def get_label_using_logits(logits, top_number=5):
    predict_y = [idx for idx in range(len(logits)) if logits[idx] >= 0.5]
    if len(predict_y) == 0: predict_y = [np.argmax(logits)]
    return predict_y

def load_data(h5_file_path, pik_file_path):
    if not os.path.exists(h5_file_path) or not os.path.exists(pik_file_path):
        raise RuntimeError('No such file!!')

    print('cache files exist, going to load in...')
    print('loading h5_file...')
    h5_file = h5py.File(h5_file_path, 'r')
    print('h5_file.keys:', h5_file.keys())
    train_X, train_y = h5_file['train_X'], h5_file['train_Y']
    vaild_X, valid_y = h5_file['vaild_X'], h5_file['valid_Y']
    test_X,  test_y  = h5_file['test_X'],  h5_file['test_Y']
    #embedding_final = h5_file['embedding']

    print('loading pickle file')
    word2index, label2index = None, None
    with open(pik_file_path, 'rb') as pkl:
        word2index,label2index = pickle.load(pkl)
    print('cache files load successful!')
    return word2index,label2index,train_X,train_y,vaild_X,valid_y,test_X,test_y

def do_eval(sess, model, eval_X, eval_y):
    test_X, test_y = eval_X[:3000], eval_y[:3000]
    num_of_data = len(test_y)
    batch_size = 1
    loss, F1, p, r = 0., 0., 0., 0.
    label_dict_confuse = {'TP':0.000001, 'FN':0.000001, 'FP':0.000001}
    for start in range(num_of_data):
        end = start + 1
        lo,logits = sess.run([model.loss_val, model.logits], 
                        feed_dict={model.input_x: test_X[start:end,:FLAGS.sentence_len], model.input_y: test_y[start:end],
                                   model.dropout_keep_prob:1.0})
        loss += lo
        pre = get_label_using_logits(logits[0])
        label = get_target_label_short(test_y[start])
#         pre = np.argsort(logits[0])[-5:]
#         label = [i for i in range(len(test_y[start])) if test_y[start][i] > 0]
        if start == 0: print('label:',label, 'predict:', pre)
        inter = len([x for x in pre if x in label])
        label_dict_confuse['TP'] += inter
        label_dict_confuse['FN'] += len(label) - inter
        label_dict_confuse['FP'] += len(pre) - inter
    print(label_dict_confuse)
    p = float(label_dict_confuse['TP'])/(label_dict_confuse['TP']+label_dict_confuse['FP'])
    r = float(label_dict_confuse['TP'])/(label_dict_confuse['TP']+label_dict_confuse['FN'])
    if p + r == 0: return loss/num_of_data, 0, 0, 0
    F1 = (2 * p * r)/(p + r)
    return loss/num_of_data, F1, p, r


In [13]:
tf.reset_default_graph()
tf.app.run()

cache files exist, going to load in...
loading h5_file...
h5_file.keys: KeysView(<HDF5 file "data.h5" (mode r)>)
loading pickle file
cache files load successful!
train_X[0:5]: [[ 832   60  256 1172 3407  516   96  138  103 1108   16    3   96  177
    22   11  672   53   18 1560 1560   15   65   12  180   10  342  173
    13  103  141  707  191   12  342  173   15   13   22   11  229  264
   163 1362  135 1249  156  156  731  115   84   10  808 1713  103  141
   229  264  788  421  103  141   12   95  316   10  808 1713  103  141
    12 2413 1227   15 1397  997   22  116  301  489   12   18  858   99
   596   98   26  646  813   10  386 1093  197  767   22   11 1179 1849
   593   84   22   11   94  102  322   60  190  220  583  355   10  153
   103  141  192  153   12   10 1244  466  116  103  141  189   58  130
    95  316   12  788  421  699   16    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0 

[2019/03/23 10:20:37] Epoch 4\Batch 200\ Train Loss:14.351
[2019/03/23 10:24:15] Epoch 4\Batch 400\ Train Loss:14.331
[2019/03/23 10:27:55] Epoch 4\Batch 600\ Train Loss:14.315
[2019/03/23 10:31:34] Epoch 4\Batch 800\ Train Loss:14.296
[2019/03/23 10:35:15] Epoch 4\Batch 1000\ Train Loss:14.280
[2019/03/23 10:38:55] Epoch 4\Batch 1200\ Train Loss:14.264
[2019/03/23 10:42:35] Epoch 4\Batch 1400\ Train Loss:14.247
run model on validation data...
label: [572, 793] predict: [23]
{'FN': 6570.000001, 'FP': 2571.000001, 'TP': 429.000001}
[2019/03/23 10:43:27] Epoch 4/ Validation Loss:14.128/ F1_score:0.086/ Precision:0.143/ Recall:0.061
[2019/03/23 10:47:07] Epoch 4\Batch 1600\ Train Loss:14.225
[2019/03/23 10:50:48] Epoch 4\Batch 1800\ Train Loss:14.206
[2019/03/23 10:54:28] Epoch 4\Batch 2000\ Train Loss:14.191
[2019/03/23 10:58:09] Epoch 4\Batch 2200\ Train Loss:14.177
[2019/03/23 11:01:49] Epoch 4\Batch 2400\ Train Loss:14.160
[2019/03/23 11:05:28] Epoch 4\Batch 2600\ Train Loss:14.144
[2

[2019/03/23 16:32:43] Epoch 10\Batch 1600\ Train Loss:13.214
[2019/03/23 16:37:06] Epoch 10\Batch 1800\ Train Loss:13.211
[2019/03/23 16:41:28] Epoch 10\Batch 2000\ Train Loss:13.213
[2019/03/23 16:45:19] Epoch 10\Batch 2200\ Train Loss:13.214
[2019/03/23 16:49:12] Epoch 10\Batch 2400\ Train Loss:13.213
[2019/03/23 16:53:07] Epoch 10\Batch 2600\ Train Loss:13.212
[2019/03/23 16:57:00] Epoch 10\Batch 2800\ Train Loss:13.211
run model on validation data...
label: [572, 793] predict: [23]
{'FN': 6411.000001, 'FP': 2414.000001, 'TP': 588.000001}
[2019/03/23 16:57:55] Epoch 10/ Validation Loss:13.181/ F1_score:0.118/ Precision:0.196/ Recall:0.084
[2019/03/23 17:03:24] Epoch 11\Batch 200\ Train Loss:13.212
[2019/03/23 17:07:11] Epoch 11\Batch 400\ Train Loss:13.209
[2019/03/23 17:14:41] Epoch 11\Batch 800\ Train Loss:13.206
[2019/03/23 17:18:25] Epoch 11\Batch 1000\ Train Loss:13.207
[2019/03/23 17:22:07] Epoch 11\Batch 1200\ Train Loss:13.207
[2019/03/23 17:25:46] Epoch 11\Batch 1400\ Train

{'FN': 6408.000001, 'FP': 2411.000001, 'TP': 591.000001}
[2019/03/23 22:37:17] Epoch 16/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019/03/23 22:42:47] Epoch 17\Batch 200\ Train Loss:13.198
[2019/03/23 22:46:26] Epoch 17\Batch 400\ Train Loss:13.194
[2019/03/23 22:50:06] Epoch 17\Batch 600\ Train Loss:13.194
[2019/03/23 22:53:45] Epoch 17\Batch 800\ Train Loss:13.192
[2019/03/23 22:57:25] Epoch 17\Batch 1000\ Train Loss:13.194
[2019/03/23 23:01:05] Epoch 17\Batch 1200\ Train Loss:13.194
[2019/03/23 23:04:45] Epoch 17\Batch 1400\ Train Loss:13.194
run model on validation data...
label: [572, 793] predict: [23]
{'FN': 6408.000001, 'FP': 2411.000001, 'TP': 591.000001}
[2019/03/23 23:05:36] Epoch 17/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019/03/23 23:09:17] Epoch 17\Batch 1600\ Train Loss:13.190
[2019/03/23 23:12:57] Epoch 17\Batch 1800\ Train Loss:13.188
[2019/03/23 23:16:35] Epoch 17\Batch 2000\ Train Loss:13.189
[2019/03/23 

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
tf.reset_default_graph()
tf.app.run()

cache files exist, going to load in...
loading h5_file...
h5_file.keys: KeysView(<HDF5 file "data.h5" (mode r)>)
loading pickle file
cache files load successful!
train_X[0:5]: [[ 832   60  256 1172 3407  516   96  138  103 1108   16    3   96  177
    22   11  672   53   18 1560 1560   15   65   12  180   10  342  173
    13  103  141  707  191   12  342  173   15   13   22   11  229  264
   163 1362  135 1249  156  156  731  115   84   10  808 1713  103  141
   229  264  788  421  103  141   12   95  316   10  808 1713  103  141
    12 2413 1227   15 1397  997   22  116  301  489   12   18  858   99
   596   98   26  646  813   10  386 1093  197  767   22   11 1179 1849
   593   84   22   11   94  102  322   60  190  220  583  355   10  153
   103  141  192  153   12   10 1244  466  116  103  141  189   58  130
    95  316   12  788  421  699   16    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0 

[2019/03/24 08:52:51] Epoch 21\Batch 5800\ Train Loss:13.193
[2019/03/24 08:53:39] Epoch 21\Batch 6000\ Train Loss:13.194
[2019/03/24 08:54:27] Epoch 21\Batch 6200\ Train Loss:13.191
[2019/03/24 08:55:14] Epoch 21\Batch 6400\ Train Loss:13.190
[2019/03/24 08:56:01] Epoch 21\Batch 6600\ Train Loss:13.188
[2019/03/24 08:56:46] Epoch 21\Batch 6800\ Train Loss:13.188
[2019/03/24 08:57:34] Epoch 21\Batch 7000\ Train Loss:13.188
run model on validation data...
label: [572, 793] predict: [23]
{'TP': 591.000001, 'FN': 6408.000001, 'FP': 2411.000001}
[2019/03/24 08:58:12] Epoch 21/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019/03/24 08:59:00] Epoch 21\Batch 7200\ Train Loss:13.187
[2019/03/24 08:59:47] Epoch 21\Batch 7400\ Train Loss:13.188
[2019/03/24 09:00:32] Epoch 21\Batch 7600\ Train Loss:13.188
[2019/03/24 09:01:20] Epoch 21\Batch 7800\ Train Loss:13.189
[2019/03/24 09:02:08] Epoch 21\Batch 8000\ Train Loss:13.189
[2019/03/24 09:02:56] Epoch 21\Batch 8200\ Tr

[2019/03/24 10:13:17] Epoch 23\Batch 1200\ Train Loss:13.188
[2019/03/24 10:14:03] Epoch 23\Batch 1400\ Train Loss:13.186
run model on validation data...
label: [572, 793] predict: [23]
{'TP': 591.000001, 'FN': 6408.000001, 'FP': 2411.000001}
[2019/03/24 10:14:40] Epoch 23/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019/03/24 10:15:28] Epoch 23\Batch 1600\ Train Loss:13.194
[2019/03/24 10:16:15] Epoch 23\Batch 1800\ Train Loss:13.193
[2019/03/24 10:17:02] Epoch 23\Batch 2000\ Train Loss:13.191
[2019/03/24 10:17:48] Epoch 23\Batch 2200\ Train Loss:13.193
[2019/03/24 10:18:34] Epoch 23\Batch 2400\ Train Loss:13.194
[2019/03/24 10:19:22] Epoch 23\Batch 2600\ Train Loss:13.196
[2019/03/24 10:20:10] Epoch 23\Batch 2800\ Train Loss:13.194
run model on validation data...
label: [572, 793] predict: [23]
{'TP': 591.000001, 'FN': 6408.000001, 'FP': 2411.000001}
[2019/03/24 10:20:47] Epoch 23/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019

[2019/03/24 11:30:14] Epoch 24\Batch 7200\ Train Loss:13.187
[2019/03/24 11:31:01] Epoch 24\Batch 7400\ Train Loss:13.188
[2019/03/24 11:31:47] Epoch 24\Batch 7600\ Train Loss:13.188
[2019/03/24 11:32:34] Epoch 24\Batch 7800\ Train Loss:13.189
[2019/03/24 11:33:21] Epoch 24\Batch 8000\ Train Loss:13.189
[2019/03/24 11:34:09] Epoch 24\Batch 8200\ Train Loss:13.189
[2019/03/24 11:34:56] Epoch 24\Batch 8400\ Train Loss:13.189
run model on validation data...
label: [572, 793] predict: [23]
{'TP': 591.000001, 'FN': 6408.000001, 'FP': 2411.000001}
[2019/03/24 11:35:36] Epoch 24/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019/03/24 11:36:24] Epoch 24\Batch 8600\ Train Loss:13.190
[2019/03/24 11:37:11] Epoch 24\Batch 8800\ Train Loss:13.191
[2019/03/24 11:37:58] Epoch 24\Batch 9000\ Train Loss:13.191
[2019/03/24 11:38:46] Epoch 24\Batch 9200\ Train Loss:13.189
[2019/03/24 11:39:31] Epoch 24\Batch 9400\ Train Loss:13.190
[2019/03/24 11:40:18] Epoch 24\Batch 9600\ Tr

[2019/03/24 12:50:23] Epoch 26\Batch 2600\ Train Loss:13.196
[2019/03/24 12:51:11] Epoch 26\Batch 2800\ Train Loss:13.194
run model on validation data...
label: [572, 793] predict: [23]
{'TP': 591.000001, 'FN': 6408.000001, 'FP': 2411.000001}
[2019/03/24 12:51:48] Epoch 26/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019/03/24 12:52:35] Epoch 26\Batch 3000\ Train Loss:13.192
[2019/03/24 12:53:21] Epoch 26\Batch 3200\ Train Loss:13.192
[2019/03/24 12:54:08] Epoch 26\Batch 3400\ Train Loss:13.193
[2019/03/24 12:54:56] Epoch 26\Batch 3600\ Train Loss:13.193
[2019/03/24 12:55:43] Epoch 26\Batch 3800\ Train Loss:13.194
[2019/03/24 12:56:30] Epoch 26\Batch 4000\ Train Loss:13.193
[2019/03/24 12:57:16] Epoch 26\Batch 4200\ Train Loss:13.193
run model on validation data...
label: [572, 793] predict: [23]
{'TP': 591.000001, 'FN': 6408.000001, 'FP': 2411.000001}
[2019/03/24 12:57:54] Epoch 26/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019

[2019/03/24 14:07:19] Epoch 27\Batch 8600\ Train Loss:13.190
[2019/03/24 14:08:07] Epoch 27\Batch 8800\ Train Loss:13.191
[2019/03/24 14:08:55] Epoch 27\Batch 9000\ Train Loss:13.191
[2019/03/24 14:09:41] Epoch 27\Batch 9200\ Train Loss:13.189
[2019/03/24 14:10:27] Epoch 27\Batch 9400\ Train Loss:13.190
[2019/03/24 14:11:13] Epoch 27\Batch 9600\ Train Loss:13.190
[2019/03/24 14:12:00] Epoch 27\Batch 9800\ Train Loss:13.190
run model on validation data...
label: [572, 793] predict: [23]
{'TP': 591.000001, 'FN': 6408.000001, 'FP': 2411.000001}
[2019/03/24 14:12:38] Epoch 27/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019/03/24 14:13:26] Epoch 27\Batch 10000\ Train Loss:13.190
[2019/03/24 14:14:13] Epoch 27\Batch 10200\ Train Loss:13.190
[2019/03/24 14:14:58] Epoch 27\Batch 10400\ Train Loss:13.190
[2019/03/24 14:15:45] Epoch 27\Batch 10600\ Train Loss:13.191
[2019/03/24 14:16:33] Epoch 27\Batch 10800\ Train Loss:13.190
[2019/03/24 14:17:21] Epoch 27\Batch 110

[2019/03/24 15:27:25] Epoch 29\Batch 4000\ Train Loss:13.193
[2019/03/24 15:28:12] Epoch 29\Batch 4200\ Train Loss:13.193
run model on validation data...
label: [572, 793] predict: [23]
{'TP': 591.000001, 'FN': 6408.000001, 'FP': 2411.000001}
[2019/03/24 15:28:50] Epoch 29/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019/03/24 15:29:38] Epoch 29\Batch 4400\ Train Loss:13.192
[2019/03/24 15:30:24] Epoch 29\Batch 4600\ Train Loss:13.191
[2019/03/24 15:31:10] Epoch 29\Batch 4800\ Train Loss:13.193
[2019/03/24 15:31:58] Epoch 29\Batch 5000\ Train Loss:13.195
[2019/03/24 15:32:45] Epoch 29\Batch 5200\ Train Loss:13.195
[2019/03/24 15:33:33] Epoch 29\Batch 5400\ Train Loss:13.192
[2019/03/24 15:34:19] Epoch 29\Batch 5600\ Train Loss:13.194
run model on validation data...
label: [572, 793] predict: [23]
{'TP': 591.000001, 'FN': 6408.000001, 'FP': 2411.000001}
[2019/03/24 15:34:58] Epoch 29/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019

[2019/03/24 16:45:36] Epoch 30\Batch 10000\ Train Loss:13.190
[2019/03/24 16:46:23] Epoch 30\Batch 10200\ Train Loss:13.190
[2019/03/24 16:47:10] Epoch 30\Batch 10400\ Train Loss:13.190
[2019/03/24 16:47:57] Epoch 30\Batch 10600\ Train Loss:13.191
[2019/03/24 16:48:44] Epoch 30\Batch 10800\ Train Loss:13.190
[2019/03/24 16:49:31] Epoch 30\Batch 11000\ Train Loss:13.189
[2019/03/24 16:50:18] Epoch 30\Batch 11200\ Train Loss:13.189
run model on validation data...
label: [572, 793] predict: [23]
{'TP': 591.000001, 'FN': 6408.000001, 'FP': 2411.000001}
[2019/03/24 16:50:56] Epoch 30/ Validation Loss:13.168/ F1_score:0.118/ Precision:0.197/ Recall:0.084
[2019/03/24 16:51:43] Epoch 30\Batch 11400\ Train Loss:13.189


SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
